<center>
    <h1>CREDIT RISK ANALYTICS</h1>
</center>

**Background:** Credit  scoring  is  the  set  of  decision  models  and  their  underlying  techniques 
that  aid  lenders  in  the  granting  of  consumer  credit.  These  techniques determine  who  will  get  credit,  how  much  credit  they  should  get,  and  what operational  strategies  will  enhance  the  profitability  of  the  borrowers  to  the lenders.  Further,  they  help  to  assess  the  risk  in  lending.  Credit  scoring  is  a dependable  assessment  of  a  person’s  credit  worthiness  since  it  is  based  on actual data.

#### Definition of Target and Outcome Window:
One of the leading banks would like to predict bad customer while customer applying for loan. This model also called as PD Models (Probability of Default)

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats


%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 8
plt.style.use("seaborn")

In [2]:
#for machine learning

import statsmodels.formula.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate

## Data Pre-Processing -
 - Missing Values Treatment - Numerical (Mean/Median imputation) and Categorical (Separate Missing Category or Merging)
 - Univariate Analysis - Outlier and Frequency Analysis

### Load Dataset

In [3]:
bankloans = pd.read_csv("Data/bankloans.csv")

In [4]:
bankloans.head()

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,176,9.3,11.359392,5.008608,1.0
1,27,1,10,6,31,17.3,1.362202,4.000798,0.0
2,40,1,15,14,55,5.5,0.856075,2.168925,0.0
3,41,1,15,14,120,2.9,2.658720,0.821280,0.0
4,24,2,2,0,28,17.3,1.787436,3.056564,1.0


In [5]:
bankloans.columns

Index(['age', 'ed', 'employ', 'address', 'income', 'debtinc', 'creddebt',
       'othdebt', 'default'],
      dtype='object')

In [6]:
#number of observations and features
bankloans.shape

(850, 9)

In [7]:
#data types in the dataframe
bankloans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       850 non-null    int64  
 1   ed        850 non-null    int64  
 2   employ    850 non-null    int64  
 3   address   850 non-null    int64  
 4   income    850 non-null    int64  
 5   debtinc   850 non-null    float64
 6   creddebt  850 non-null    float64
 7   othdebt   850 non-null    float64
 8   default   700 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 59.9 KB


<big>
- Checking for missing values

In [8]:
#check for any column has missing values
bankloans.isnull().any()

age         False
ed          False
employ      False
address     False
income      False
debtinc     False
creddebt    False
othdebt     False
default      True
dtype: bool

In [ ]:
#check for number of missing values
bankloans.isnull().sum()

In [ ]:
#Segregating the numeric and categorical variable names

numeric_var_names = [key for key in dict(bankloans.dtypes) if dict(bankloans.dtypes)[key] in ['float64', 'int64', 'float32', 'int32']]
catgorical_var_names = [key for key in dict(bankloans.dtypes) if dict(bankloans.dtypes)[key] in ['object']]


In [ ]:
numeric_var_names

In [ ]:
#splitting the data set into two sets - existing customers and new customers

bankloans_existing = bankloans.loc[bankloans.default.isnull() == 0]
bankloans_new = bankloans.loc[bankloans.default.isnull() == 1]

In [ ]:
bankloans_existing.describe(percentiles=[.25,0.5,0.75,0.90,0.95])

<big>
    - Checking for Outliers

In [ ]:
sns.boxplot(y = "age",data=bankloans_existing)
plt.title("Box-Plot of age")
plt.show()

In [ ]:
sns.boxplot(y = "employ",data=bankloans_existing)
plt.title("Box-Plot of employee tenure")
plt.show()

In [ ]:
sns.boxplot(y = "income",data=bankloans_existing)
plt.title("Box-Plot of employee income")
plt.show()

In [ ]:
sns.boxplot(y = "debtinc",data=bankloans_existing)
plt.title("Box-Plot of employee debt to income ratio")
plt.show()

In [ ]:
sns.boxplot(y = "creddebt",data=bankloans_existing)
plt.title("Box-Plot of Credit to debit ratio")
plt.show()

In [ ]:
income_minlimit = bankloans_existing["income"].quantile(0.75) + 1.5 * (bankloans_existing["income"].quantile(0.75) - bankloans_existing["income"].quantile(0.25))
income_minlimit

In [ ]:
def outlier_capping(x):
    """A funtion to remove and replace the outliers for numerical columns"""
    x = x.clip(upper=x.quantile(0.95))
    
    return(x)

In [ ]:
#outlier treatment
bankloans_existing = bankloans_existing.apply(lambda x: outlier_capping(x))

In [ ]:
##Correlation Matrix
bankloans_existing.corr()

In [ ]:
#Visualize the correlation using seaborn heatmap

sns.heatmap(bankloans_existing.corr(),annot=True,fmt="0.2f",cmap="coolwarm")
plt.show()

In [ ]:
bankloans_existing.shape

In [ ]:
bankloans_new.shape

In [ ]:
#Indicator variable unique types

bankloans_existing['default'].value_counts()

In [ ]:
bankloans_existing['default'].value_counts().plot.bar()
plt.xlabel("default")
plt.ylabel("count")
plt.title("Distribution of default")
plt.show()

In [ ]:
#percentage of unique types in indicator variable

round(bankloans_existing['default'].value_counts()/bankloans_existing.shape[0] * 100,3)

## Data Exploratory Analysis
- Bivariate Analysis - Numeric(TTest)/ Categorical(Chisquare)
- Bivariate Analysis - Visualization
- Variable Reduction - Multicollinearity

In [ ]:
## performing the independent t test on numerical variables

tstats_df = pd.DataFrame()

for eachvariable in numeric_var_names:
    tstats = stats.ttest_ind(bankloans_existing.loc[bankloans_existing["default"] == 1,eachvariable],bankloans_existing.loc[bankloans_existing["default"] == 0, eachvariable],equal_var=False)
    temp = pd.DataFrame([eachvariable, tstats[0], tstats[1]]).T
    temp.columns = ['Variable Name', 'T-Statistic', 'P-Value']
    tstats_df = pd.concat([tstats_df, temp], axis=0, ignore_index=True)
    
tstats_df =  tstats_df.sort_values(by = "P-Value").reset_index(drop = True)

In [ ]:
tstats_df

### Bi-Variate Analysis

In [ ]:
def BivariateAnalysisPlot(segment_by):
    """A funtion to analyze the impact of features on the target variable"""
    
    fig, ax = plt.subplots(ncols=1,figsize = (10,8))
    
    #boxplot
    sns.boxplot(x = 'default', y = segment_by, data=bankloans_existing)
    plt.title("Box plot of "+segment_by)
    
    
    plt.show()
    

In [ ]:
BivariateAnalysisPlot("age")

In [ ]:
BivariateAnalysisPlot("ed")

In [ ]:
BivariateAnalysisPlot("employ")

In [ ]:
BivariateAnalysisPlot("address")

In [ ]:
BivariateAnalysisPlot("income")

In [ ]:
BivariateAnalysisPlot("debtinc")

In [ ]:
BivariateAnalysisPlot("creddebt")

In [ ]:
BivariateAnalysisPlot("othdebt")

### Multi Collinearity Check

In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
features = "+".join(bankloans_existing.columns.difference(["default"]))

In [ ]:
features

In [ ]:
#perform vif

a, b = dmatrices(formula_like= 'default ~ ' + features,data=bankloans_existing,return_type="dataframe")
vif = pd.DataFrame()

vif["VIF Factor"] = [variance_inflation_factor(b.values, i) for i in range(b.shape[1])]
vif["Features"] = b.columns

In [ ]:
vif

### Observations
----
<big>
- There are 850 observations and 9 features in the data set
- All 9 features are numerical in nature
- There are no missing values in the data set
- Out of 850 customers data, 700 are existing customers and 150 are new customers
- In the 700 existing customers, 517 customers are tagged as non defaulters and remaining 183 are tagged as defaulters
- The data is highly imbalanced
- From VIF check, found out that the correlation between the variables is within the acceptable limits

## Model Building and Model Diagnostics

   - Logistic Regression
   - Decision Tree classifier
---
**Model Diagnostics**

- Train and Test split
- Significance of each Variable
- Gini and ROC / Concordance analysis
- Classification Table Analysis - Accuracy

### Logistic Regression

In [ ]:
featurecolumns = bankloans_existing.columns.difference(['default'])
featurecolumns

In [ ]:
#Train and test split

train_X,test_X,train_y,test_y = train_test_split(bankloans_existing[featurecolumns],
                                                 bankloans_existing['default'], stratify = bankloans_existing['default'], test_size = 0.2, random_state = 123)

In [ ]:
train_X.shape

In [ ]:
test_X.shape

In [ ]:
round(train_y.value_counts()/train_y.shape[0] * 100,3)

In [ ]:
## Model Building

logreg = LogisticRegression()
logreg.fit(train_X,train_y)

In [ ]:
#Features and their coefficients

coefficient_df =  pd.DataFrame({'Features' : pd.Series(featurecolumns),
                        "Coefficients" : pd.Series(logreg.coef_[0])})
coefficient_df

In [ ]:
logreg.intercept_

### Model Performance 
- Test data set

#### Metrics

- Recall: Ratio of the total number of correctly classified positive examples divide to the total number of positive examples. High Recall indicates the class is correctly recognized
- Precision: To get the value of precision we divide the total number of correctly classified positive examples by the total number of predicted positive examples. High Precision indicates an example labeled as positive is indeed positive

In [ ]:
#Predicting the test cases
bankloans_test_pred_log = pd.DataFrame({'actual':test_y, 'predicted': logreg.predict(test_X)})
bankloans_test_pred_log = bankloans_test_pred_log.reset_index()
bankloans_test_pred_log.head()

In [ ]:
#creating a confusion matrix

cm_logreg = metrics.confusion_matrix(bankloans_test_pred_log.actual,
                                    bankloans_test_pred_log.predicted,labels = [1,0])
cm_logreg

In [ ]:
sns.heatmap(cm_logreg,annot=True, fmt=".2f", cmap="coolwarm",
            xticklabels = ["Default", "Not Default"] , yticklabels = ["Default", "Not Default"])
plt.title("Confusion Matrix for Test data")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()

In [ ]:
#find precision score

prec_score = metrics.precision_score(bankloans_test_pred_log.actual, bankloans_test_pred_log.predicted)
print("Precision score :", round(prec_score,3))

In [ ]:
#find the overall accuracy of model

acc_score = metrics.accuracy_score(bankloans_test_pred_log.actual,bankloans_test_pred_log.predicted)
print("Accuracy of model :", round(acc_score,3))

In [ ]:
bankloans_test_pred_log.actual.value_counts()

In [ ]:
print(metrics.classification_report(bankloans_test_pred_log.actual, bankloans_test_pred_log.predicted))

#### Inference
-----

<big>
Overall test accuracy is 80%. But it is not a good measure. There are lot of cases which are default and the model has predicted them as not default. The objective of the model is to identify the customers who will default, so that the bank can intervene and act.This might be the case as the default model assumes people with more than 0.5 probability will not default. 
</big>

### Find the optimum cutoff value

In [ ]:
#probabilty of prediction

predict_prob_df = pd.DataFrame(logreg.predict_proba(test_X))
predict_prob_df.head()

In [ ]:
bankloans_test_pred_log = pd.concat([bankloans_test_pred_log, predict_prob_df], axis = 1)
bankloans_test_pred_log.columns = ['index', 'actual', 'predicted', 'default_0','default_1']

bankloans_test_pred_log.head()

In [ ]:
#find the auc score

auc_score = metrics.roc_auc_score(bankloans_test_pred_log.actual, bankloans_test_pred_log.default_1)
round(auc_score,4)

In [ ]:
#Draw a roc curve

fpr, tpr, thresholds = metrics.roc_curve(bankloans_test_pred_log.actual, bankloans_test_pred_log.default_1, 
                                         drop_intermediate= False)


plt.plot(fpr, tpr , label = 'ROC curve (area = %0.2f)' % auc_score)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic Curve')
plt.legend(loc="lower right")
plt.show()

<big>
- Cutoff would be optimum where specificity and sensitivity would be maximum for the given cutoff

In [ ]:
##TPR - Sensitivity
##1-FPR - Specificity

i = np.arange(len(tpr))

roc_like_df = pd.DataFrame({'falsepositiverate' : pd.Series(fpr, index=i),'sensitivity' : pd.Series(tpr, index = i), 
              'specificity' : pd.Series(1-fpr, index = i),'cutoff' : pd.Series(thresholds, index = i)})
roc_like_df['total'] = roc_like_df['sensitivity'] + roc_like_df['specificity']

In [ ]:
roc_like_df[roc_like_df['total']==roc_like_df['total'].max()]

In [ ]:
plt.subplots(figsize=(10,6))
plt.scatter(roc_like_df['cutoff'], roc_like_df['sensitivity'], marker='*', label='Sensitivity')
plt.scatter(roc_like_df['cutoff'], roc_like_df['specificity'], marker='*', label='Specificity')
plt.scatter(roc_like_df['cutoff'], roc_like_df['falsepositiverate'], marker='*', label='FPR')
plt.title('For each cutoff, pair of sensitivity and FPR is plotted for ROC')
plt.legend()

plt.show()

In [ ]:
#Predicting with new cut-off probability
bankloans_test_pred_log['new_labels'] = bankloans_test_pred_log['default_1'].map( lambda x: 1 if x >= 0.224326 else 0 )

bankloans_test_pred_log.head()

In [ ]:
#creating a confusion matrix

cm_logreg = metrics.confusion_matrix(bankloans_test_pred_log.actual,
                                    bankloans_test_pred_log.new_labels,labels = [1,0])
cm_logreg

In [ ]:
sns.heatmap(cm_logreg,annot=True, fmt=".2f", cmap="coolwarm",
            xticklabels = ["Default", "Not Default"] , yticklabels = ["Default", "Not Default"])
plt.title("Confusion Matrix for Test data")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()

In [ ]:
#classification report 

print(metrics.classification_report(bankloans_test_pred_log.actual,bankloans_test_pred_log.new_labels))

In [ ]:
#intuitively the ability of the classifier to find all the positive samples

recall_score = metrics.recall_score(bankloans_test_pred_log.actual, bankloans_test_pred_log.new_labels)
print("recall_score:", round(recall_score , 3))

In [ ]:
#find the overall accuracy of model

acc_score = metrics.accuracy_score(bankloans_test_pred_log.actual,bankloans_test_pred_log.new_labels)
print("Accuracy of model :", round(acc_score,3))

#### Inference
-----

<big>
Even though the overall accuracy of the model is reduced from 80% to 75% by taking optimum cutoff as 0.224, Model performance i.e recall score (ability of the model to find all the positive samples - find all the default customers) has increased from 54% to 89%. The drawback of changing the cutoff value can be seen in drastic drop of precision score (ability of model not to label non default customers as default customers) from 67% to 52%. 

</big>

- We have a choice to make depending on the value we place on the true positives and our tolerance for false postivies, in practical the cutoff values depends on the business decision values.

## Decision Tree Classifier

In [ ]:
#make a pipeline for decision tree model

pipelines = {
    "dtclass": make_pipeline(DecisionTreeClassifier(random_state=100))
}


In [ ]:
#To check the accuracy of the pipeline
scores = cross_validate(pipelines['dtclass'],train_X,train_y,return_train_score=True)
scores['test_score'].mean()

#### Cross-Validation and Hyper Parameters Tuning
Cross Validation is the process of finding the best combination of parameters for the model by traning and evaluating the model for each combination of the parameters
- Declare a hyper-parameters to fine tune the Decision Tree Classifier

**Decision Tree is a greedy alogritum it searches the entire space of possible decision trees. so we need to find a optimum parameter(s) or criteria for stopping the decision tree at some point. We use the hyperparameters to prune the decision tree**

In [ ]:
#list of tunable hyper parameters for decision tree classifier pipeline

pipelines['dtclass'].get_params().keys()

In [ ]:
decisiontree_hyperparameters = {
    'decisiontreeclassifier__max_depth' : np.arange(3, 10),
    'decisiontreeclassifier__max_features' : np.arange(3, 8),
    'decisiontreeclassifier__min_samples_split' : np.arange(2, 15),
    "decisiontreeclassifier__min_samples_leaf" : np.arange(1,3)
}

### Decision Tree classifier with gini index

#### Fit and tune models with cross-validation

Now that we have our <code style="color:steelblue">pipelines</code> and <code style="color:steelblue">hyperparameters</code> dictionaries declared, we're ready to tune our models with cross-validation.
- We are doing 5 fold cross validation

In [ ]:
#Create a cross validation object from decision tree classifier and it's hyperparameters

dtclass_model = GridSearchCV(pipelines['dtclass'],decisiontree_hyperparameters,cv=5, n_jobs=-1)

In [ ]:
#fit the model

dtclass_model.fit(train_X, train_y)

In [ ]:
#display the best parameters for decision tree model

dtclass_model.best_params_

In [ ]:
#best score for the model
dtclass_model.best_score_

In [ ]:
#In Pipeline we can use the string names to get the decisiontreeclassifer

dtclass_best_model = dtclass_model.best_estimator_.named_steps['decisiontreeclassifier']
dtclass_best_model

### Model Performance Evaluation
- On Test Data

In [ ]:
#Predicting the test cases
bankloans_test_pred_dtclass = pd.DataFrame({'actual':test_y, 'predicted': dtclass_best_model.predict(test_X)})
bankloans_test_pred_dtclass = bankloans_test_pred_dtclass.reset_index()
bankloans_test_pred_dtclass.head()

In [ ]:
#creating a confusion matrix

cm_dtclass = metrics.confusion_matrix(bankloans_test_pred_dtclass.actual,
                                    bankloans_test_pred_dtclass.predicted,labels = [1,0])
cm_dtclass

In [ ]:
sns.heatmap(cm_dtclass,annot=True, fmt=".2f",
            xticklabels = ["Default", "Not Default"] , yticklabels = ["Default", "Not Default"])
plt.title("Confusion Matrix for Test data")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()

In [ ]:
#probabilty of prediction

predict_prob_df = pd.DataFrame(dtclass_best_model.predict_proba(test_X))
predict_prob_df.head()

In [ ]:
bankloans_test_pred_dtclass = pd.concat([bankloans_test_pred_dtclass, predict_prob_df], axis = 1)
bankloans_test_pred_dtclass.columns = ['index', 'actual', 'predicted', 'default_0','default_1']

bankloans_test_pred_dtclass.head()

In [ ]:
#find the auc score

auc_score = metrics.roc_auc_score(bankloans_test_pred_dtclass.actual, bankloans_test_pred_dtclass.default_1)
round(auc_score,4)

In [ ]:
#plotting the roc curve

fpr, tpr, thresholds = metrics.roc_curve(bankloans_test_pred_dtclass.actual, bankloans_test_pred_dtclass.default_1,
                                         drop_intermediate=False)

plt.plot(fpr, tpr, label = "ROC Curve (Area = %0.4f)" % auc_score)
plt.plot([1,0],[1,0],'k--')

plt.ylabel("True Positive Rate")
plt.xlabel("False Positive Rate or [1 - True Negative Rate]")

plt.legend(loc = "lower right")
plt.show()

In [ ]:
#find precision score

prec_score = metrics.precision_score(bankloans_test_pred_dtclass.actual, bankloans_test_pred_dtclass.predicted)
print("Precision score :", round(prec_score,3))

In [ ]:
#find the overall accuracy of model

acc_score = metrics.accuracy_score(bankloans_test_pred_dtclass.actual,bankloans_test_pred_dtclass.predicted)
print("Accuracy of model :", round(acc_score,3))

In [ ]:
#classification report

print(metrics.classification_report(bankloans_test_pred_dtclass.actual,bankloans_test_pred_dtclass.predicted))

### Visualization of Decision Tree
- Dependencies 
    - Need to install graphviz (conda install pydot graphviz)
    - Set the environment path variable to graphviz folder

In [ ]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus as pdot
from io import StringIO
import graphviz

In [ ]:
#writing the dot data
dot_data = StringIO()

In [ ]:
#export the decision tree along with the feature names into a dot file format

export_graphviz(dtclass_best_model,out_file=dot_data,filled=True,special_characters=True,rounded=True,
                feature_names=train_X.columns.values,class_names = ["No","Yes"])

In [ ]:
#make a graph from dot file

graph = pdot.graph_from_dot_data(dot_data.getvalue())

In [ ]:
Image(graph.create_png())

## Model Selection and Business Insights
___
<big>
- Based on the F1-score (harmonic mean of precision and recall), logistic model with f1 score (for positive labels - default customers) of 0.66 is giving better results than decision tree model with f1 score of 0.44. So we will use the logistic regression model to predict the credit worthiness of the customers 
    
-We will Predict the credit risk for remainimg 150 customers using the logistic model with cutoff as 0.224 

In [ ]:
#probability for new customers

new_cust_prob = pd.DataFrame(logreg.predict_proba(bankloans_new[featurecolumns]))
new_cust_prob.columns = ["prob_default_0", "prob_default_1"]
new_cust_prob.index = bankloans_new.index

In [ ]:
new_cust_prob.head()

In [ ]:
bankloans_new_predicted = pd.concat([bankloans_new,new_cust_prob],axis=1)
bankloans_new_predicted.head()

In [ ]:
#using the cutoff value we will predict the default

bankloans_new_predicted['predicted_default'] = bankloans_new_predicted["prob_default_1"].apply(lambda x: 1 if x > 0.224 else 0)

In [ ]:
bankloans_new_predicted.head()

In [ ]:
#Model Prediction

bankloans_new_predicted.predicted_default.value_counts()

In [ ]:
#Model Prediction

bankloans_new_predicted.predicted_default.value_counts().plot.bar()

plt.ylabel("Count")
plt.xlabel("Default")
plt.show()

## Insights
---

- Out of 150 new customers, model has predicted that 85 customers are not going to default on the bank loan and remaining 65 customes would most likely default on the loan.

### Model Performance Validation

- KS Chart
- Lift and Gain Chart

we will use the concept of decile analysis for these validations

In [ ]:
#For train data

train_predict = pd.DataFrame({'actual': train_y.reset_index(drop = True), 
                              'prob':  pd.DataFrame(logreg.predict_proba(train_X))[1]})
train_predict['predicted'] = train_predict['prob'].apply(lambda x: 1 if x > 0.224 else 0)
train_predict.head()

In [ ]:
#For test data

test_predict = pd.DataFrame({'actual': test_y.reset_index(drop = True), 
                              'prob':  pd.DataFrame(logreg.predict_proba(test_X))[1]})
test_predict['predicted'] = test_predict['prob'].apply(lambda x: 1 if x > 0.224 else 0)
test_predict.head()

<big>
    Split the data into different deciles - train and test data

In [ ]:
#splitting the train data into different deciles

train_predict['Deciles']=pd.qcut(train_predict['prob'],10, labels=False)
train_predict.head()

In [ ]:
#splitting the test data into different deciles

test_predict['Deciles']=pd.qcut(test_predict['prob'],10, labels=False)
test_predict.head()

In [ ]:
#sumation of deciles for train data

train_predict[['Deciles','actual']].groupby(train_predict.Deciles).sum().sort_index(ascending=False)

In [ ]:
#sumation of deciles for test data

test_predict[['Deciles','actual']].groupby(test_predict.Deciles).sum().sort_index(ascending=False)

In [ ]:
train_predict[['Deciles','actual']].groupby(train_predict.Deciles).count().sort_index(ascending=False)

In [ ]:
test_predict[['Deciles','actual']].groupby(test_predict.Deciles).count().sort_index(ascending=False)

In [ ]:
from IPython.display import Image

### KS & Lift, Gain Chart
- Training dataset
- Testing dataset

In [ ]:
#For Training data set
Image(filename="Images/KS-Traindata.png",width=600)

In [ ]:
#For Test data set
Image(filename="Images/KS-Testdata.png",width=600)

In [ ]:
##Lift Chart 
Image(filename="Images/LiftChart.png",width=500)

In [ ]:
##Lift Chart 
Image(filename="Images/GainsChart.png",width=500)

### Observations
---
<big>
- Gain chart tells % of targets (events) covered at a given decile level. In the current case, we can say that we can identify 90% of the defaulters who are likely to default on the loan by just analyzing 50% of the total customers.
- Lift chart measures how much better one can expect to do with the predictive model comparing without a model. In the current model, cummulative lift for top two deciles is 2.7, means that by selecting 20% of the records based on the model. One can expect 2.7 times the total number of defaulters to be found than the randomly selecting 20% of the data without a model.

## Finally, let's save the winning model.
- We need to save your prediction models to file, and then restore them in order to reuse your previous work to: test your model on new data, compare multiple models, or anything else.
- Pickle is the standard way of serializing objects in Python.Pickle operation to serialize your machine learning algorithms and save the serialized format to a file.
- Later you can load this file to deserialize your model and use it to make new predictions.

In [ ]:
import pickle

Let's save the winning <code style="color:steelblue">Logistic Model</code> object into a pickle file.

In [ ]:
with open('OutPutModel/final_model.pkl', 'wb') as f:
    pickle.dump(logreg, f)

In [ ]:
# some time later...
 
# load the model from disk - use to classify the default customers directly
#loaded_model = pickle.load(open('OutPutModel/final_model.pkl', 'rb'))
